In [12]:
!pip install transformers peft bitsandbytes trl deepeval optimum auto-gptq langchain torch flask
!pip install --upgrade langchain

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

#Quantize 4bits

In [15]:
compute_dtypes = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtypes,
)

In [4]:
# from google.colab import userdata
# userdata.get('Llama-3-8B')

In [17]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [18]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B

model_id = "meta-llama/Meta-Llama-3-8B"
access_token = "#"

# Pass the access token to the tokenizer as well
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_id, token = access_token, quantization_config = quant_config,  device_map={"": 0})

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=500)
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
prompt_template = PromptTemplate(
    input_variables=["product_name", "product_description", "interest_level", "user_name", "tone"],
    template="""
Write a personalized marketing email in a paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices, discount codes, or dates:

Product Name: {product_name}
Product Description: {product_description}
Interest Level: {interest_level}
User Name: {user_name}
Email Tone: {tone}

Email Content in Paragraph:
Subject: Exclusive {product_name} Offer Just for You!

Dear {user_name},
"""
)

email_chain = LLMChain(llm=hf, prompt=prompt_template)

In [23]:
import re
# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))  # Default

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email in paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices of the product, discount codes, or dates:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Interest Level: {interest_level}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content in Paragraph:\nSubject: Exclusive {product_name} Offer Just for You!\n\n\n"
        f"Dear {user_name},\n\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    while True:  # Loop for validation
        # Generate the output
        outputs = model.generate(
            inputs.input_ids.to('cuda'),
            max_length=500,
            min_length=200,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
        )

        # Decode and process the email
        email = tokenizer.decode(outputs[0], skip_special_tokens=True)
        email_content_start = email.find("Subject:")
        generated_email = email[email_content_start:].strip()


        # Remove any existing sign-offs
        sign_offs = ["Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best", "money would be refunded", "company would be liable", "product can be returned after used", "simply use the discount"]
        for sign_off in sign_offs:
            if sign_off in generated_email:
                generated_email = generated_email.split(sign_off)[0].strip()

        # Guard against generated coupon codes
        forbidden_phrases = ["code", "coupon code", "discount code", "offer code", "Discount Offer", "retailed price", "retailed rate","discounted price", "discounted rate", "price", "product price", " 30-day money-back guarantee"]
        for phrase in forbidden_phrases:
            if phrase in generated_email:
                generated_email = generated_email.split(phrase)[0].strip()


        # Filter and remove unwanted elements
        generated_email = re.sub(r'http\S+', '', generated_email)
        generated_email = re.sub(r'www\.\S+', '', generated_email)
        generated_email = re.sub(r'<.*?>', '', generated_email)
        generated_email = re.sub(r'\d+% off', '', generated_email)

        # Append template values with dynamic coupon code and discount offer
        generated_email += (
            f"\n\nUse the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.\n"
            f"\nBest regards,\n"
            f"{sender_name}\n"
            f"{company_name}\n"
            f"Visit us: {website_url}\n"
        )

        # Validation check
        if is_valid_email(generated_email):
            return generated_email
        else:
            print("Email not valid. Regenerating...")



In [24]:
def is_valid_email(email):
    # Customization based on my requirement
    forbidden_phrases = ["money would be refunded", "company would be liable",
                         "product can be returned after used", "simply use the discount", "sales", "promotions", "discounts", "prices", "discount codes", "dates", "regular price", "Sincerely", "The SuperWidgets Team", "Yours sincerely", "Name of Company"]
    if any(phrase in email for phrase in forbidden_phrases):
        return False
    if len(email.split()) < 100 or len(email.split()) > 200:
        return False
    return True



In [25]:
# example for the generation
product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

Email not valid. Regenerating...
Subject: Exclusive SuperWidget Offer Just for You!


Dear John Doe,


Thank you for expressing interest in our SuperWidget product. We are excited to offer you a special discount on our product. Our SuperWidget can help you increase your productivity and boost your efficiency. It is a one-of-a-kind tool that is designed to meet your specific needs and requirements. Our product is highly customizable and can be tailored to fit your unique needs. We guarantee that our Super Widget will exceed your expectations. Our team of experts is always available to assist you with any questions or concerns you may have. We would love to hear from you soon. Please do not hesitate to contact us if you have any questions. We look forward to hearing from you. Thank you again for your interest in SuperWidget. We hope you enjoy using our product as much as we enjoy creating it.

Use the coupon code 'NEPAL30' to get 30% off. Valid till 1 DECEMBER 2024.

Best regards,
Binit 